In [1]:
%load_ext autoreload
%autoreload 2

In [9]:
import neuralop
import torch

from denoising import (
    Environment,
    ModelRegistry,
)
from config import model_load_configs

In [3]:
print('torch ' + torch.__version__)
print('neuralop ' + neuralop.__version__)

torch 2.6.0+cu124
neuralop 1.0.2


In [4]:
# загружаем переменные окружения
env = Environment(_env_file='../env')
print(f'Used device: {env.device}, weights_dir: {env.weights}')

Used device: cuda:2, weights_dir: /home/d.nesterov/denoising-fno/notebooks


# Инициализация датасетов

In [5]:
from denoising import (
    make_bsd_dset_config,
    make_fno_dset_config,
    make_load_params,
)
from denoising.data import DatasetRegistry

In [6]:
mri_root = env.data / 'MRI/IXI_0_1/255'
bsd_root = env.data / 'BSDS300-horizontal-synthetic'
sidd_root = env.data / 'SIDD_Small_sRGB_Only'

mri_sketch_load_params = make_load_params('sketch', [145, 145], 'float32')
mri_image_load_params = make_load_params('image', [145, 145], 'float32')
mri_gt_load_params = make_load_params('gt', [255, 255], 'float32')
pm_load_params = [mri_sketch_load_params, mri_image_load_params]
gt_load_params = [mri_sketch_load_params, mri_gt_load_params]

sidd_noisy_load_params = make_load_params('noisy', [512, 512, 3], 'uint8')
sidd_gt_load_params = make_load_params('gt', [512, 512, 3], 'uint8')
sidd_load_params = [sidd_noisy_load_params, sidd_gt_load_params]

datasets_configs = {
    # MRI datasets
    'mri_pm_test': make_fno_dset_config(
        mri_root,
        env.data / 'MRI/lists/IXI_0_1/train_pmLR_gibbsnoiseLR_val.csv',
        pm_load_params,
    ),
    'mri_gt_test': make_fno_dset_config(
        mri_root,
        env.data / 'MRI/lists/IXI_0_1/train_gtLR_gibbsnoiseLR_val.csv',
        gt_load_params,
    ),
    # BSD datasets
    'bsd_synth_0.01_test': make_bsd_dset_config(bsd_root, 0.01, 'test'),
    # SIDD datasets, patches
    'sidd_test': make_fno_dset_config(
        sidd_root / 'val',
        sidd_root / 'patches_val.csv',
        sidd_load_params,
        normalize=True,
    ),
}

In [7]:
dataset_registry = DatasetRegistry()
dataset_registry.load(datasets_configs, verbose=True)
dataset_registry

Got n_samples = 2093  in dataset mri_pm_test         with sample size = torch.Size([1, 145, 145])
Got n_samples = 2093  in dataset mri_gt_test         with sample size = torch.Size([1, 145, 145])
Got n_samples = 77    in dataset bsd_synth_0.01_test with sample size = torch.Size([1, 321, 481])
Got n_samples = 3008  in dataset sidd_test           with sample size = torch.Size([3, 512, 512])


DatasetRegistry(['mri_pm_test', 'mri_gt_test', 'bsd_synth_0.01_test', 'sidd_test'])

# Инициализация моделей

In [10]:
model_registry = ModelRegistry()
model_registry.load(model_load_configs, random_seed=env.random_seed, device='cpu', verbose=True)
model_registry

Loaded  model mri-fno-neuralop with n_parameters = 2010449
Loaded  model mri-fno-custom   with n_parameters = 2010449
Loaded  model mri-fno-spectral2d with n_parameters = 2010449
Loaded  model mri-hno          with n_parameters = 4476513
Loaded  model sidd-fno-run2    with n_parameters = 2011091
Loaded  model sidd-fno-run3    with n_parameters = 2011091
Loaded  model sidd-fno-run4    with n_parameters = 2028627
Loaded  model sidd-hno-run5    with n_parameters = 1786579
Loaded  model bsd-fno          with n_parameters = 2010449


ModelRegistry(['mri-fno-neuralop', 'mri-fno-custom', 'mri-fno-spectral2d', 'mri-hno', 'sidd-fno-run2', 'sidd-fno-run3', 'sidd-fno-run4', 'sidd-hno-run5', 'bsd-fno'])

# Инференс и подсчет метрик

In [11]:
from torchmetrics.image import StructuralSimilarityIndexMeasure as SSIM

from denoising import Metrics, make_metric_config

In [12]:
metrics = Metrics([make_metric_config('ssim', SSIM, {'data_range': 1.0})])

In [13]:
print(f'Available datasets: {dataset_registry}')
print(f'Available models:   {model_registry}')
print(f'Metrics to compute: {metrics}')

Available datasets: DatasetRegistry(['mri_pm_test', 'mri_gt_test', 'bsd_synth_0.01_test', 'sidd_test'])
Available models:   ModelRegistry(['mri-fno-neuralop', 'mri-fno-custom', 'mri-fno-spectral2d', 'mri-hno', 'sidd-fno-run2', 'sidd-fno-run3', 'sidd-fno-run4', 'sidd-hno-run5', 'bsd-fno'])
Metrics to compute: Metrics(['ssim'])


In [14]:
from denoising import Evaluator

In [15]:
ev = Evaluator(model_registry, dataset_registry, metrics, device=env.device)

In [20]:
m_mri = ev.evaluate(
    ['mri-fno-neuralop', 'mri-fno-custom', 'mri_pm_test'],
    'mri_gt_test',
    batch_size=128,
)
m_mri

Computing predicts for ('mri-fno-neuralop', 'mri_gt_test') from model


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [00:04<00:00,  3.65it/s]


Saved predicts for ('mri-fno-neuralop', 'mri_gt_test') to cache
Computing predicts for ('mri-fno-custom', 'mri_gt_test') from model


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [00:04<00:00,  3.75it/s]


Saved predicts for ('mri-fno-custom', 'mri_gt_test') to cache


{'mri-fno-neuralop_ssim': tensor(0.9333),
 'mri-fno-custom_ssim': tensor(0.8398),
 'mri_pm_test_ssim': tensor(0.9360)}

In [23]:
m_sidd = ev.evaluate(
    ['sidd-fno-run4'],
    # ['sidd-fno-run4', 'sidd-hno-run5'],
    'sidd_test',
    batch_size=32,
)
m_sidd

Retrieving predicts for ('sidd-fno-run4', 'sidd_test') from cache


{'sidd-fno-run4_ssim': tensor(0.5579)}

In [27]:
m_bsd = ev.evaluate(['bsd-fno'], 'bsd_synth_0.01_test', batch_size=128)
m_bsd

Computing predicts for ('bsd-fno', 'bsd_synth_0.01_test') from model


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.63s/it]


Saved predicts for ('bsd-fno', 'bsd_synth_0.01_test') to cache


{'bsd-fno_ssim': tensor(0.1189)}

In [22]:
m_mri

{'mri-fno-neuralop_ssim': tensor(0.9333),
 'mri-fno-custom_ssim': tensor(0.8398),
 'mri_pm_test_ssim': tensor(0.9360)}

In [21]:
m_sidd

{'sidd-fno-run2_ssim': tensor(0.8911),
 'sidd-fno-run3_ssim': tensor(0.8871),
 'sidd-fno-run4_ssim': tensor(0.9105)}

In [31]:
m_bsd

{'bsd-fno_ssim': tensor(0.1189)}

In [24]:
model_registry['sidd-fno-run4']

FNO(
  (positional_embedding): GridEmbeddingND()
  (fno_blocks): FNOBlocks(
    (convs): ModuleList(
      (0-15): 16 x SpectralConv2D(
        (weight): TuckerTensor(shape=(16, 16, 32, 17), rank=(13, 13, 26, 14))
      )
    )
    (fno_skips): ModuleList(
      (0-15): 16 x Flattened1dConv(
        (conv): Conv1d(16, 16, kernel_size=(1,), stride=(1,), bias=False)
      )
    )
    (channel_mlp): ModuleList(
      (0-15): 16 x ChannelMLP(
        (fcs): ModuleList(
          (0): Conv1d(16, 8, kernel_size=(1,), stride=(1,))
          (1): Conv1d(8, 16, kernel_size=(1,), stride=(1,))
        )
      )
    )
    (channel_mlp_skips): ModuleList(
      (0-15): 16 x SoftGating()
    )
  )
  (lifting): ChannelMLP(
    (fcs): ModuleList(
      (0): Conv1d(5, 128, kernel_size=(1,), stride=(1,))
      (1): Conv1d(128, 16, kernel_size=(1,), stride=(1,))
    )
  )
  (projection): ChannelMLP(
    (fcs): ModuleList(
      (0): Conv1d(16, 32, kernel_size=(1,), stride=(1,))
      (1): Conv1d(32, 3, k